# Stated Preferences Survey - Datenanalyse

Dieses Notebook zeigt, wie die gesammelten Daten aus der SP-Umfrage analysiert werden können.

## Inhalt
1. Daten laden und vorbereiten
2. Deskriptive Statistik
3. Visualisierungen
4. Vorbereitung für Discrete Choice Modelle
5. Beispiel: Multinomial Logit (MNL) Modell

In [ ]:
# Import von Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2

# Styling für Plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display-Einstellungen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Daten laden

In [ ]:
# Lade die CSV-Daten
df = pd.read_csv('../data/responses.csv')

print(f"Anzahl Beobachtungen: {len(df)}")
print(f"Anzahl Befragte: {df['respondent_id'].nunique()}")
print(f"Anzahl Szenarien pro Person: {len(df) / df['respondent_id'].nunique():.0f}")

# Erste Zeilen anzeigen
df.head()

## 2. Deskriptive Statistik

### 2.1 Soziodemografie

In [ ]:
# Verteilung Alter
print("Altersverteilung:")
print(df.groupby('respondent_id')['age'].first().value_counts())
print()

# Verteilung Geschlecht
print("Geschlechterverteilung:")
print(df.groupby('respondent_id')['gender'].first().value_counts())
print()

# Verteilung Wohnort
print("Wohnort:")
print(df.groupby('respondent_id')['location'].first().value_counts())

### 2.2 Mobilitätsverhalten

In [ ]:
# Hauptverkehrsmittel
print("Hauptverkehrsmittel:")
print(df.groupby('respondent_id')['main_mode'].first().value_counts())
print()

# Durchschnittliche Pendelstrecke
avg_commute = df.groupby('respondent_id')['commute_distance_km'].first().astype(float).mean()
print(f"Durchschnittliche Pendelstrecke: {avg_commute:.1f} km")
print()

# ÖPNV-Nutzung
print("ÖPNV-Nutzungshäufigkeit:")
print(df.groupby('respondent_id')['pt_frequency'].first().value_counts())

### 2.3 Choice-Verteilung

In [ ]:
# Verteilung der Choices
print("Gewählte Alternativen:")
choice_counts = df['choice'].value_counts()
print(choice_counts)
print()
print("Prozentuale Verteilung:")
print(choice_counts / len(df) * 100)

## 3. Visualisierungen

In [ ]:
# Visualisierung der Choice-Verteilung
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Choice-Verteilung
choice_counts.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Verteilung der gewählten Alternativen', fontsize=14)
axes[0].set_xlabel('Alternative')
axes[0].set_ylabel('Anzahl')
axes[0].tick_params(axis='x', rotation=0)

# Altersverteilung
age_data = df.groupby('respondent_id')['age'].first().value_counts().sort_index()
age_data.plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Altersverteilung der Befragten', fontsize=14)
axes[1].set_xlabel('Altersgruppe')
axes[1].set_ylabel('Anzahl Befragte')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Einstellungen visualisieren
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

attitudes = [
    ('env_awareness', 'Umweltbewusstsein', axes[0, 0]),
    ('tech_affinity', 'Technikaffinität', axes[0, 1]),
    ('autonomous_openness', 'Offenheit für autonomes Fahren', axes[1, 0]),
    ('price_sensitivity', 'Preissensibilität', axes[1, 1])
]

for col, title, ax in attitudes:
    attitude_data = df.groupby('respondent_id')[col].first().value_counts().sort_index()
    attitude_data.plot(kind='bar', ax=ax, color='teal')
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('Wert (1=niedrig, 5=hoch)')
    ax.set_ylabel('Anzahl Befragte')
    ax.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 4. Datenvorbereitung für Discrete Choice Modelle

Für Discrete Choice Modelle müssen die Daten in einem speziellen Format vorliegen:
- Eine Zeile pro Alternative pro Choice-Situation
- Eine binäre Choice-Variable (0/1) für jede Alternative

In [ ]:
# Erstelle Long-Format für Discrete Choice Modelle
def prepare_choice_data(df):
    """
    Transformiert die Daten in ein Format für Discrete Choice Modelle.
    Jede Zeile wird zu 4 Zeilen (eine pro Alternative A, B, C, none).
    """
    rows = []
    
    for idx, row in df.iterrows():
        # Alternative A
        rows.append({
            'respondent_id': row['respondent_id'],
            'scenario_id': row['scenario_id'],
            'alternative': 'A',
            'mode': row['alt_a_mode'],
            'time': row['alt_a_time'],
            'cost': row['alt_a_cost'],
            'reliability': row['alt_a_reliability'],
            'co2': row['alt_a_co2'],
            'chosen': 1 if row['choice'] == 'A' else 0,
            'age': row['age'],
            'gender': row['gender'],
            'env_awareness': row['env_awareness'],
            'tech_affinity': row['tech_affinity']
        })
        
        # Alternative B
        rows.append({
            'respondent_id': row['respondent_id'],
            'scenario_id': row['scenario_id'],
            'alternative': 'B',
            'mode': row['alt_b_mode'],
            'time': row['alt_b_time'],
            'cost': row['alt_b_cost'],
            'reliability': row['alt_b_reliability'],
            'co2': row['alt_b_co2'],
            'chosen': 1 if row['choice'] == 'B' else 0,
            'age': row['age'],
            'gender': row['gender'],
            'env_awareness': row['env_awareness'],
            'tech_affinity': row['tech_affinity']
        })
        
        # Alternative C
        rows.append({
            'respondent_id': row['respondent_id'],
            'scenario_id': row['scenario_id'],
            'alternative': 'C',
            'mode': row['alt_c_mode'],
            'time': row['alt_c_time'],
            'cost': row['alt_c_cost'],
            'reliability': row['alt_c_reliability'],
            'co2': row['alt_c_co2'],
            'chosen': 1 if row['choice'] == 'C' else 0,
            'age': row['age'],
            'gender': row['gender'],
            'env_awareness': row['env_awareness'],
            'tech_affinity': row['tech_affinity']
        })
        
        # Alternative "Keine davon"
        rows.append({
            'respondent_id': row['respondent_id'],
            'scenario_id': row['scenario_id'],
            'alternative': 'none',
            'mode': 'none',
            'time': 0,
            'cost': 0,
            'reliability': 0,
            'co2': 'none',
            'chosen': 1 if row['choice'] == 'none' else 0,
            'age': row['age'],
            'gender': row['gender'],
            'env_awareness': row['env_awareness'],
            'tech_affinity': row['tech_affinity']
        })
    
    return pd.DataFrame(rows)

# Erstelle das transformierte Dataset
df_long = prepare_choice_data(df)
print(f"Anzahl Zeilen im Long-Format: {len(df_long)}")
df_long.head(12)  # Zeige die ersten 12 Zeilen (3 Szenarien)

## 5. Multinomial Logit (MNL) Modell - Konzept

### Theoretischer Hintergrund

Das Multinomial Logit Modell basiert auf der Random Utility Theory:

$$U_{ni} = V_{ni} + \epsilon_{ni}$$

Wobei:
- $U_{ni}$ = Nutzen der Alternative $i$ für Person $n$
- $V_{ni}$ = Deterministischer (beobachtbarer) Nutzen
- $\epsilon_{ni}$ = Zufallskomponente

Der deterministische Nutzen wird oft als lineare Funktion modelliert:

$$V_{ni} = \beta_{time} \cdot time_i + \beta_{cost} \cdot cost_i + \beta_{reliability} \cdot reliability_i + ...$$

Die Wahrscheinlichkeit, dass Alternative $i$ gewählt wird:

$$P_{ni} = \frac{e^{V_{ni}}}{\sum_{j \in C_n} e^{V_{nj}}}$$

### Implementierung mit statsmodels (vereinfachtes Beispiel)

In [ ]:
# HINWEIS: Für richtige Discrete Choice Modelle sollte man spezialisierte
# Bibliotheken wie pylogit, biogeme oder xlogit verwenden.
# Hier zeigen wir ein vereinfachtes Beispiel mit statsmodels.

from statsmodels.discrete.discrete_model import MNLogit
from statsmodels.tools import add_constant

# Für dieses Beispiel erstellen wir ein vereinfachtes Modell
# das nur die Haupteffekte schätzt (ohne "none"-Option)

# Filtere "none" Alternativen raus für dieses Beispiel
df_model = df_long[df_long['alternative'] != 'none'].copy()

# Erstelle numerische Variablen
df_model['time_norm'] = df_model['time'] / 10  # Normalisierung für bessere Konvergenz
df_model['cost_norm'] = df_model['cost']
df_model['reliability_norm'] = df_model['reliability'] / 100

# Dummy-Variablen für CO2
co2_dummies = pd.get_dummies(df_model['co2'], prefix='co2')
df_model = pd.concat([df_model, co2_dummies], axis=1)

# Wähle Features für das Modell
feature_cols = ['time_norm', 'cost_norm', 'reliability_norm']
X = df_model[feature_cols]

# Erstelle Choice-Variable (0, 1, 2 für A, B, C)
alt_mapping = {'A': 0, 'B': 1, 'C': 2}
df_model['alt_numeric'] = df_model['alternative'].map(alt_mapping)

# Reshape für MNLogit
# Wir müssen die Daten so umformen, dass jede Choice-Situation eine Zeile hat
print("\nHINWEIS: Für ein vollständiges Discrete Choice Modell sollten Sie")
print("spezialisierte Bibliotheken wie 'pylogit' oder 'biogeme' verwenden.")
print("\nDiese ermöglichen:")
print("- Korrekte Berücksichtigung der Choice-Set-Struktur")
print("- Alternative-spezifische Konstanten")
print("- Mixed Logit Modelle")
print("- Nested Logit Modelle")
print("- und vieles mehr...")

## 6. Interpretation und nächste Schritte

### Typische Fragestellungen:

1. **Welche Attribute beeinflussen die Verkehrsmittelwahl am stärksten?**
   - Reisezeit, Kosten, Zuverlässigkeit, CO₂-Ausstoß
   
2. **Gibt es Unterschiede zwischen soziodemografischen Gruppen?**
   - Z.B. unterschiedliche Preissensibilität nach Alter
   
3. **Wie groß ist die Zahlungsbereitschaft (Willingness to Pay)?**
   - Z.B. Wie viel würden Befragte für kürzere Reisezeiten zahlen?
   
4. **Wie attraktiv ist autonomes Ridesharing?**
   - Marktanteile unter verschiedenen Szenarien

### Empfohlene nächste Schritte:

1. Installieren Sie `pylogit` oder `biogeme` für professionelle DCM-Analyse:
   ```
   pip install pylogit
   ```

2. Spezifizieren Sie ein vollständiges MNL-Modell mit:
   - Alternative-spezifischen Konstanten
   - Interaktionseffekten (z.B. Kosten × Einkommen)
   - Ggf. Mixed Logit für Präferenzheterogenität

3. Berechnen Sie Willingness-to-Pay Werte:
   $$WTP = -\frac{\beta_{attribut}}{\beta_{cost}}$$

4. Führen Sie Policy-Simulationen durch:
   - Was wäre, wenn ÖPNV günstiger/schneller würde?
   - Wie würden sich Marktanteile verschieben?

## Export für weitere Analysen

In [ ]:
# Speichere das Long-Format für weitere Analysen
df_long.to_csv('../data/responses_long_format.csv', index=False)
print("Daten im Long-Format gespeichert unter: ../data/responses_long_format.csv")